# Exercise : 서비스 이탈예측 데이터(Classification)
- 출처 : https://www.datamanim.com/dataset/03_dataq/typetwo.html#id3

- 데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
- 데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형
  
- X_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv
- y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv
- X_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv
- y_test(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv

---

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Data Load & EDA

In [2]:
import pandas as pd

X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
X_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv")

In [3]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6499, 12), (3501, 12), (6499, 2), (3501, 2))

In [3]:
display(X_train.head())
display(y_train.head())

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 609.4+ KB


In [5]:
display(X_train.describe())
display(X_test.describe())

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,6.499000e+03,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000
mean,1.569157e+07,650.396830,38.957070,5.041545,76836.581068,1.519772,0.708878,0.514387,100346.564524
std,7.187584e+04,96.618957,10.502803,2.891779,62407.570894,0.578975,0.454314,0.499831,57944.655305
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000
25%,1.562949e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50907.565000
50%,1.569181e+07,651.000000,37.000000,5.000000,97560.160000,1.000000,1.000000,1.000000,100496.840000
75%,1.575358e+07,718.000000,44.000000,8.000000,127844.690000,2.000000,1.000000,1.000000,150480.155000
max,1.581566e+07,850.000000,92.000000,10.000000,238387.560000,4.000000,1.000000,1.000000,199970.740000


,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,3.501000e+03,3501.000000,3501.000000,3501.000000,3501.000000,3501.000000,3501.000000,3501.000000,3501.000000
mean,1.568977e+07,650.773779,38.856327,4.959440,75834.890751,1.549557,0.699229,0.516424,99614.417586
std,7.204360e+04,96.730345,10.461094,2.892564,62382.215537,0.586187,0.458659,0.499802,56700.884072
min,1.556571e+07,376.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,91.750000
25%,1.562648e+07,583.000000,32.000000,2.000000,0.000000,1.000000,0.000000,0.000000,51229.170000
50%,1.568917e+07,652.000000,37.000000,5.000000,96196.240000,2.000000,1.000000,1.000000,99770.300000
75%,1.575260e+07,717.000000,44.000000,7.000000,127286.040000,2.000000,1.000000,1.000000,146411.410000
max,1.581569e+07,850.000000,85.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000


# 2. Data Preprocessing

In [6]:
X_train = X_train.drop(["CustomerId", "Surname"], axis=1)

X_test_id = X_test["CustomerId"]
X_test = X_test.drop(["CustomerId", "Surname"], axis=1)

display(X_train.head())
display(X_test.head())

y_train = y_train["Exited"]
print(y_train.value_counts())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,709,France,Female,32,2,0.00,2,0,0,109681.29
4,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,802,France,Female,60,3,92887.06,1,1,0,39473.63
1,602,France,Female,56,3,115895.22,3,1,0,4176.17
2,801,France,Female,32,4,75170.54,1,1,1,37898.50
3,693,Spain,Female,34,10,107556.06,2,0,0,154631.35
4,592,France,Female,62,5,0.00,1,1,1,100941.57


0    5175
1    1324
Name: Exited, dtype: int64


## Checking missing values

In [7]:
print(X_train.isnull().sum())
print(X_test.isnull().sum())

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64


## Encoding

In [8]:
for col in X_train.select_dtypes(object).columns.tolist():
    print('---------')
    print(col)
    print(X_train[col].nunique())
    print(X_train[col].value_counts())
    
for col in X_test.select_dtypes(object).columns.tolist():
    print('---------')
    print(col)
    print(X_test[col].nunique())
    print(X_test[col].value_counts())

---------
Geography
3
France     3227
Germany    1650
Spain      1622
Name: Geography, dtype: int64
---------
Gender
4
Male      3485
Female    2861
 male       80
female      73
Name: Gender, dtype: int64
---------
Geography
3
France     1787
Germany     859
Spain       855
Name: Geography, dtype: int64
---------
Gender
4
Male      1839
Female    1562
 male       53
female      47
Name: Gender, dtype: int64


In [9]:
X_train["Gender"] = X_train["Gender"].apply(str.strip).apply(str.lower)
X_test["Gender"] = X_test["Gender"].apply(str.strip).apply(str.lower)

print(X_train["Gender"].value_counts())
print('---------')
print(X_test["Gender"].value_counts())

male      3565
female    2934
Name: Gender, dtype: int64
---------
male      1892
female    1609
Name: Gender, dtype: int64


In [10]:
from sklearn.preprocessing import LabelEncoder

c_cols = X_train.select_dtypes(object).columns

for col in c_cols :
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    
display(X_train.head())
display(X_test.head())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,791,1,0,35,7,52436.20,1,1,0,161051.75
1,705,1,1,42,8,166685.92,2,1,1,55313.51
2,543,0,0,31,4,138317.94,1,0,0,61843.73
3,709,0,0,32,2,0.00,2,0,0,109681.29
4,714,1,0,36,1,101609.01,2,1,1,447.73


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,802,0,0,60,3,92887.06,1,1,0,39473.63
1,602,0,0,56,3,115895.22,3,1,0,4176.17
2,801,0,0,32,4,75170.54,1,1,1,37898.50
3,693,2,0,34,10,107556.06,2,0,0,154631.35
4,592,0,0,62,5,0.00,1,1,1,100941.57


## Scaling

In [11]:
from sklearn.preprocessing import StandardScaler

print("before scaling \n", X_train.head())
print('---------')

n_cols = X_train.select_dtypes(exclude="object").columns
    
scaler = StandardScaler()
X_train[n_cols] = scaler.fit_transform(X_train[n_cols])
X_test[n_cols] = scaler.transform(X_test[n_cols])

print("after scaling \n ", X_train.head())

before scaling 
    CreditScore  Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          791          1       0   35       7   52436.20              1   
1          705          1       1   42       8  166685.92              2   
2          543          0       0   31       4  138317.94              1   
3          709          0       0   32       2       0.00              2   
4          714          1       0   36       1  101609.01              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               0        161051.75  
1          1               1         55313.51  
2          0               0         61843.73  
3          0               0        109681.29  
4          1               1           447.73  
---------
after scaling 
     CreditScore  Geography    Gender       Age    Tenure   Balance  \
0     1.455346   0.298362 -1.102300 -0.376792  0.677301 -0.391014   
1     0.565183   0.298362  0.907194  0.289748  1.023136  1.439829   
2   

# 3. Splitting Data

In [12]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size = 0.2, random_state = 1)

print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(5199, 10) (1300, 10) (5199,) (1300,)


# 4. Modeling

In [13]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, roc_auc_score

In [14]:
rf = RandomForestClassifier(random_state=1)
gb = GradientBoostingClassifier(random_state=1)
xgb = XGBClassifier(random_state=1)

models = [rf, gb, xgb]

In [15]:
# Hyperparameter tuning

from sklearn.model_selection import GridSearchCV

params = {"n_estimators":[100,200,300,400,500], "max_depth":[1,2,3,4,5]}

# best_models = []

for model in models:
    gs = GridSearchCV(model, param_grid=params, cv = 5, scoring="roc_auc", n_jobs=4)
    gs.fit(X_tr, y_tr)
    
    print(f"="*10)
    print(f"model : {model}")
    print(f"params : {gs.best_params_}")
    print(f"score : {gs.best_score_}") # 최고 정확도

model : RandomForestClassifier(random_state=1)
params : {'max_depth': 5, 'n_estimators': 100}
score : 0.8498457318108361
model : GradientBoostingClassifier(random_state=1)
params : {'max_depth': 3, 'n_estimators': 100}
score : 0.8599718678843535
model : XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ..

In [16]:
model_gb = GradientBoostingClassifier(random_state=1, max_depth=3, n_estimators=100)
model_gb.fit(X_tr, y_tr) # 학습

GradientBoostingClassifier(random_state=1)

In [17]:
# model score : train
predict_tr_label = model_gb.predict(X_tr)
predict_tr_proba = model_gb.predict_proba(X_tr)[:, 1]

# model score : Validation
predict_val_label = model_gb.predict(X_val)
predict_val_proba = model_gb.predict_proba(X_val)[:, 1]


print('train accuracy :', accuracy_score(y_tr, predict_tr_label))
print('validation accuracy :', accuracy_score(y_val, predict_val_label))

print(f"="*10)

print('train auc :', roc_auc_score(y_tr, predict_tr_proba))
print('validation auc :', roc_auc_score(y_val, predict_val_proba))

train accuracy : 0.8768994037314868
validation accuracy : 0.8638461538461538
train auc : 0.8947276165190934
validation auc : 0.8626597393127337


In [18]:
# 최종 모델로 test data 최종 예측(평가)
test_predict = model_gb.predict(X_test)
test_predict_prob = model_gb.predict_proba(X_test)

print(f"accuracy_score:", accuracy_score(y_test["Exited"], test_predict))
print(f"roc_auc_score:", roc_auc_score(y_test["Exited"], test_predict_prob[:,1]))

accuracy_score: 0.8643244787203657
roc_auc_score: 0.8611898116753629


# 5. Saving and Submission

In [19]:
submit = pd.DataFrame({"CustomerId": X_test_id, "Exited" : test_predict_prob[:,1]})
print(submit.head())

submit.to_csv("result.csv", index = False)

check = pd.read_csv("result.csv")
check.head()

   CustomerId    Exited
0    15601012  0.898722
1    15734762  0.990010
2    15586757  0.067135
3    15590888  0.107688
4    15726087  0.442673


,CustomerId,Exited
0,15601012,0.898722
1,15734762,0.990010
2,15586757,0.067135
3,15590888,0.107688
4,15726087,0.442673
